# Preprocessing

### This worksheet will consit of the the clenaing code for the reviews dataste

In [2]:
# ----------------------------------------------------------
# 1. Import Libraries
# ----------------------------------------------------------
import pandas as pd
import numpy as np

# ----------------------------------------------------------
# 2. Load the Listings Data
# ----------------------------------------------------------



listings_path = r"C:\Users\emman\Documents\Airbnb_Project\data\listings.csv"
reviews_path  = r"C:\Users\emman\Documents\Airbnb_Project\data\reviews.csv"

# Load the data
listings = pd.read_csv(listings_path)
reviews  = pd.read_csv(reviews_path)

print("Listings loaded:", listings.shape)
print("Reviews loaded:", reviews.shape)


Listings loaded: (45886, 79)
Reviews loaded: (1747347, 6)


In [108]:
listings['host_id'].unique()
print([listings['host_id'] == 107434423])

[0        False
1        False
2        False
3        False
4        False
         ...  
45881    False
45882    False
45883    False
45884    False
45885    False
Name: host_id, Length: 45886, dtype: bool]


In [102]:
listings['id'].nunique()
reviews['listing_id'].nunique()


33385

In [3]:
# Step 1 — Get ID sets
listings_ids = set(listings['id'])
reviews_ids  = set(reviews['listing_id'])

# Step 2 — Intersection
matched_ids = listings_ids.intersection(reviews_ids)
print("Listings with reviews:", len(matched_ids))

# Step 3 — Filter listings
matched_listings = listings[listings['id'].isin(matched_ids)].copy()
print("Matched listings shape:", matched_listings.shape)


Listings with reviews: 33385
Matched listings shape: (33385, 79)


In [4]:
listings_ids = set(listings['id'])
reviews_ids = set(reviews['listing_id'])

matched_ids = listings_ids.intersection(reviews_ids)
len(matched_ids)

33385

In [5]:
matched_listings.head()
matched_listings.info()
matched_listings.isna().mean().sort_values(ascending=False)


<class 'pandas.core.frame.DataFrame'>
Index: 33385 entries, 0 to 45865
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            33385 non-null  int64  
 1   listing_url                                   33385 non-null  object 
 2   scrape_id                                     33385 non-null  int64  
 3   last_scraped                                  33385 non-null  object 
 4   source                                        33385 non-null  object 
 5   name                                          33385 non-null  object 
 6   description                                   32999 non-null  object 
 7   neighborhood_overview                         17083 non-null  object 
 8   picture_url                                   33385 non-null  object 
 9   host_id                                       33385 non-null  int6

calendar_updated                                1.000000
license                                         0.684709
neighborhood_overview                           0.488303
neighbourhood                                   0.488303
host_about                                      0.417792
                                                  ...   
calculated_host_listings_count                  0.000000
calculated_host_listings_count_entire_homes     0.000000
calculated_host_listings_count_private_rooms    0.000000
calculated_host_listings_count_shared_rooms     0.000000
reviews_per_month                               0.000000
Length: 79, dtype: float64

In [5]:
reviews = reviews[['listing_id', 'date', 'comments']].copy()



In [6]:
reviews = reviews.dropna(subset=['comments'])



In [7]:
reviews = reviews.drop_duplicates(subset=['listing_id', 'comments'])


In [8]:
reviews['date'] = pd.to_datetime(reviews['date'], errors='coerce')
reviews['date'].describe()


count                          1743409
mean     2022-01-27 20:15:35.268315648
min                2009-05-26 00:00:00
25%                2019-10-30 00:00:00
50%                2022-10-19 00:00:00
75%                2024-06-15 00:00:00
max                2025-09-02 00:00:00
Name: date, dtype: object

In [9]:
cutoff = reviews['date'].max() - pd.Timedelta(days=365)
recent_reviews = reviews[reviews['date'] >= cutoff].copy()

print("Recent reviews:", len(recent_reviews))
print("Recent listings:", recent_reviews['listing_id'].nunique())




Recent reviews: 356304
Recent listings: 24140


In [17]:
matched_reviews = recent_reviews[recent_reviews['listing_id'].isin(matched_ids)].copy()
matched_reviews.head(5)

,listing_id,date,comments
43,2708,2024-12-01,The accommodation is ideally located in West H...
44,2708,2025-07-04,Wonderful host! Thank you so much.
155,181511,2025-04-19,Here to report that all the other positive rev...
156,181511,2025-06-30,This place truly felt like home. Mornings were...
202,6931,2024-12-19,Charles is a great host and always tries to ma...


In [12]:

matched_reviews.to_csv(
    r"C:\Users\emman\Documents\Airbnb_Project\data\cleaned_reviews.csv",
    index=False
)

In [11]:
clean_listings = pd.read_csv(r"C:\Users\emman\Documents\Airbnb_Project\data\cleaned_listings_df.csv")






In [15]:
clean_listings['number_of_reviews'].mean()
clean_listings['number_of_reviews'].describe()


count    43596.000000
mean        39.094215
std         84.726878
min          0.000000
25%          0.000000
50%          6.000000
75%         37.000000
max       1325.000000
Name: number_of_reviews, dtype: float64

In [12]:
# 1. Filter for ratings around ~3.0
bottom_cluster = clean_listings[
    clean_listings['review_scores_rating'].between(2.8, 3.4)
].copy()

# 2. Keep listings with enough reviews
bottom_cluster = bottom_cluster[
    bottom_cluster['number_of_reviews'] >= 5
].copy()

# NEW: 3. Remove listings with missing price
bottom_cluster = bottom_cluster.dropna(subset=['price']).copy()

# 4. Sort by lowest rating
worst_sorted = bottom_cluster.sort_values('review_scores_rating')

# 5. Remove duplicate hosts
worst_unique_hosts = worst_sorted.drop_duplicates('host_id')

# 6. Select Bottom 5
bottom_5 = worst_unique_hosts.head(5)





In [ ]:
clean_listings.columns.tolist()


['id',
 'name',
 'host_id',
 'host_name',
 'host_is_superhost',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'neighborhood_overview',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bedrooms',
 'beds',
 'bathrooms_text',
 'price',
 'number_of_reviews',
 'review_scores_rating',
 'reviews_per_month']

In [20]:
# ------------------------------------------
# STEP 1 — Setup
# ------------------------------------------

bad_id = "47332448"   # The listing with only 1 review

# Ensure string IDs match
matched_reviews['listing_id'] = matched_reviews['listing_id'].astype(str)
bottom_cluster['id'] = bottom_cluster['id'].astype(str)
# Remove Mandarin or any listing with Chinese characters
bottom_cluster = bottom_cluster[~bottom_cluster['name'].str.contains(r'[\u4e00-\u9fff]', na=False)]

# ------------------------------------------
# STEP 2 — Get actual review counts per listing
# ------------------------------------------

review_counts = matched_reviews['listing_id'].value_counts()

# Only keep listings with >= 5 real reviews
valid_review_ids = review_counts[review_counts >= 5].index

# ------------------------------------------
# STEP 3 — Filter candidates from your bottom cluster
# ------------------------------------------

candidates = bottom_cluster[
    bottom_cluster['id'].isin(valid_review_ids)
].copy()

# Remove any replacement that shares the same host as existing bottom_5
existing_hosts = bottom_5[bottom_5['id'] != bad_id]['host_id']
candidates = candidates[~candidates['host_id'].isin(existing_hosts)]


# Sort by lowest rating (your original logic)
candidates = candidates.sort_values('review_scores_rating')

# ------------------------------------------
# STEP 4 — Pick first valid replacement
# ------------------------------------------

replacement_row = candidates.head(1)

print("Replacement listing found:")
display(replacement_row)

# ------------------------------------------
# STEP 5 — Build new bottom_5
# ------------------------------------------

# Remove the bad listing
new_bottom_5 = bottom_5[bottom_5['id'] != bad_id].copy()

# Add the replacement
new_bottom_5 = pd.concat([new_bottom_5, replacement_row], ignore_index=True)

# Ensure unique IDs
new_bottom_5 = new_bottom_5.drop_duplicates(subset='id')

# Reset index
new_bottom_5 = new_bottom_5.reset_index(drop=True)

print("Updated Bottom 5:")
display(new_bottom_5)


Replacement listing found:


,id,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,neighborhood_overview,latitude,longitude,...,room_type,accommodates,bedrooms,beds,bathrooms_text,price,number_of_reviews,review_scores_rating,reviews_per_month,num_recent_reviews


Updated Bottom 5:


,id,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,neighborhood_overview,latitude,longitude,...,room_type,accommodates,bedrooms,beds,bathrooms_text,price,number_of_reviews,review_scores_rating,reviews_per_month,num_recent_reviews
0,1279389366908702099,The Creative Nest Hub,540076227.0,Four Gems,f,Pomona,Other Cities,NaN,34.072176,-117.743459,...,Entire home/apt,4.0,1.0,2.0,1 bath,$79.00,6.0,3.0,0.65,6.0
1,1354621532086945799,Cozy 2nd-Floor Studio Sleeps 2,668460439.0,Mohammad,f,Beverly Hills,Other Cities,This studio apartment is perfectly located jus...,34.059543,-118.389206,...,Entire home/apt,2.0,0.0,1.0,1 bath,$108.00,8.0,3.0,1.66,8.0
2,1446867008477628010,Spacious private br and own ba,109414810.0,James,t,Alhambra,Other Cities,NaN,34.089370,-118.125170,...,Private room,2.0,1.0,1.0,1 private bath,$43.00,5.0,3.2,2.05,5.0
3,1328494094282574396,Cozy Studio by Rodeo Drive,660630470.0,Ariella,f,Beverly Hills,Other Cities,"Located in the heart of Beverly Hills, this st...",34.062631,-118.411154,...,Entire home/apt,2.0,1.0,1.0,1 bath,$126.00,5.0,3.4,0.68,5.0
4,1469837763308127204,Tiki Hut Love Shack. 10min walk to everything,366965961.0,Daria,f,Marina del Rey,Unincorporated Areas,NaN,33.976580,-118.452450,...,Entire home/apt,2.0,0.0,1.0,1 bath,$83.00,5.0,3.4,3.85,5.0


In [21]:
# Load Top 5 listings
top_5_path = r"C:\Users\emman\Documents\Airbnb_Project\results\Top 5 Listings.csv"
top_5 = pd.read_csv(top_5_path)
top_5['id'] = top_5['id'].astype(str)

# Ensure review IDs are string
matched_reviews['listing_id'] = matched_reviews['listing_id'].astype(str)

# Filter reviews for Top 5 listings
top5_reviews = matched_reviews[matched_reviews['listing_id'].isin(top_5['id'])]

# Merge listing metadata into reviews
top5_full = top5_reviews.merge(
    top_5,
    left_on='listing_id',
    right_on='id',
    how='left'
)

# Save final merged dataset
output_path = r"C:\Users\emman\Documents\Airbnb_Project\results\top_5_with_reviews.csv"
top5_full.to_csv(output_path, index=False)

top5_full.head()


,listing_id,date,comments,id,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,...,property_type,room_type,accommodates,bedrooms,beds,bathrooms_text,price,number_of_reviews,review_scores_rating,reviews_per_month
0,16069685,2024-09-02,We stay here every time we have to be in Whitt...,16069685,Hummingbird Haven,104690415,Jennifer,f,Whittier,Other Cities,...,Entire guesthouse,Entire home/apt,2,1,1,1 bath,$131.00,505,5,4.76
1,16069685,2024-09-08,Jenny is the epitome of a gracious host. The c...,16069685,Hummingbird Haven,104690415,Jennifer,f,Whittier,Other Cities,...,Entire guesthouse,Entire home/apt,2,1,1,1 bath,$131.00,505,5,4.76
2,16069685,2024-09-10,Thank you so much for gave us a great experien...,16069685,Hummingbird Haven,104690415,Jennifer,f,Whittier,Other Cities,...,Entire guesthouse,Entire home/apt,2,1,1,1 bath,$131.00,505,5,4.76
3,16069685,2024-09-13,Jenny is a very friendly host. <br/>her house ...,16069685,Hummingbird Haven,104690415,Jennifer,f,Whittier,Other Cities,...,Entire guesthouse,Entire home/apt,2,1,1,1 bath,$131.00,505,5,4.76
4,16069685,2024-09-19,Hummingbird Haven was just that! The stress of...,16069685,Hummingbird Haven,104690415,Jennifer,f,Whittier,Other Cities,...,Entire guesthouse,Entire home/apt,2,1,1,1 bath,$131.00,505,5,4.76


In [22]:
# =====================================================
# STEP 0 — Ensure ID types match
# =====================================================

clean_listings['id'] = clean_listings['id'].astype(str)
top_5['id'] = top_5['id'].astype(str)
matched_reviews['listing_id'] = matched_reviews['listing_id'].astype(str)

# =====================================================
# STEP 1 — Count recent reviews per listing
# =====================================================

recent_counts = (
    matched_reviews['listing_id']
    .value_counts()
    .rename('num_recent_reviews')
    .reset_index()
    .rename(columns={'index': 'listing_id'})
)


recent_counts = recent_counts.rename(columns={'listing_id': 'id'})


In [23]:


# =====================================================
# STEP 2 — Build bottom_cluster AND keep the 'id' column
# =====================================================

bottom_cluster = clean_listings[
    clean_listings['review_scores_rating'].between(2.8, 3.4)
].copy()

bottom_cluster = bottom_cluster[bottom_cluster['number_of_reviews'] >= 5].copy()
bottom_cluster = bottom_cluster.dropna(subset=['price']).copy()

# REMOVE Mandarin names
bottom_cluster = bottom_cluster[
    ~bottom_cluster['name'].str.contains(r'[\u4e00-\u9fff]', na=False)
].copy()

# Make sure ID column is present and string type
bottom_cluster['id'] = bottom_cluster['id'].astype(str)

# =====================================================
# STEP 3 — Merge 
# =====================================================

bottom_cluster = bottom_cluster.merge(
    recent_counts,
    on='id',
    how='left'
)

# Only keep listings with >=5 recent reviews
bottom_cluster = bottom_cluster[bottom_cluster['num_recent_reviews'] >= 5].copy()

# =====================================================
# STEP 4 — Sort, dedupe hosts, select bottom 5
# =====================================================

worst_sorted = bottom_cluster.sort_values('review_scores_rating')
worst_unique_hosts = worst_sorted.drop_duplicates('host_id')

bottom_5 = worst_unique_hosts.head(5).reset_index(drop=True)

print("Updated Bottom 5:")
display(bottom_5[['id','name','review_scores_rating','num_recent_reviews']])

# =====================================================
# STEP 5 — Sampling Function (only id + comments)
# =====================================================

def sample_comments(df, n):
    return df.groupby('listing_id').apply(
        lambda x: x.sample(n=n, replace=False)[['listing_id', 'comments']]
        if len(x) >= n else x[['listing_id', 'comments']]
    ).reset_index(drop=True)

# =====================================================
# STEP 6 — Sample TOP 5 (10 each)
# =====================================================

top5_reviews = matched_reviews[
    matched_reviews['listing_id'].isin(top_5['id'])
]

top5_sampled = sample_comments(top5_reviews, n=10)

# =====================================================
# STEP 7 — Sample BOTTOM 5 (5 each)
# =====================================================

bottom5_reviews = matched_reviews[
    matched_reviews['listing_id'].isin(bottom_5['id'])
]

bottom5_sampled = sample_comments(bottom5_reviews, n=5)

# =====================================================
# STEP 8 — Save CSVs
# =====================================================

top5_sampled.to_csv(
    r"C:\Users\emman\Documents\Airbnb_Project\results\top_5_sampled_reviews.csv",
    index=False
)

bottom5_sampled.to_csv(
    r"C:\Users\emman\Documents\Airbnb_Project\results\bottom_5_sampled_reviews.csv",
    index=False
)

# =====================================================
# STEP 9 — Confirmation
# =====================================================

print("\nTOP 5 Sample Counts:")
print(top5_sampled['listing_id'].value_counts())

print("\nBOTTOM 5 Sample Counts:")
print(bottom5_sampled['listing_id'].value_counts())

print("\nBottom 5 and sampling completed successfully.")


Updated Bottom 5:


,id,name,review_scores_rating,num_recent_reviews
0,1279389366908702099,The Creative Nest Hub,3.0,6.0
1,1354621532086945799,Cozy 2nd-Floor Studio Sleeps 2,3.0,8.0
2,1446867008477628010,Spacious private br and own ba,3.2,5.0
3,1328494094282574396,Cozy Studio by Rodeo Drive,3.4,5.0
4,1469837763308127204,Tiki Hut Love Shack. 10min walk to everything,3.4,5.0



TOP 5 Sample Counts:
listing_id
16069685              10
16842869              10
38582617              10
43723586              10
546505210695128661    10
Name: count, dtype: int64

BOTTOM 5 Sample Counts:
listing_id
1279389366908702099    5
1328494094282574396    5
1354621532086945799    5
1446867008477628010    5
1469837763308127204    5
Name: count, dtype: int64

Bottom 5 and sampling completed successfully.


C:\Users\emman\AppData\Local\Temp\ipykernel_26100\2835652450.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('listing_id').apply(
C:\Users\emman\AppData\Local\Temp\ipykernel_26100\2835652450.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('listing_id').apply(


In [24]:
matched_reviews['listing_id'] = matched_reviews['listing_id'].astype(str)

review_counts = (
    matched_reviews.groupby("listing_id")
    .size()
    .rename("total_review_count")
    .reset_index()
)

review_counts
top_ids = [
    "16069685",
    "16842869",
    "38582617",
    "43723586",
    "546505210695128661"
]

bottom_ids = [
    "1279389366908702099",
    "1328494094282574396",
    "1354621532086945799",
    "1446867008477628010",
    "1469837763308127204"
]

top_counts = review_counts[review_counts["listing_id"].isin(top_ids)]
bottom_counts = review_counts[review_counts["listing_id"].isin(bottom_ids)]

print("TOP 5 — Total Reviews per Listing:")
print(top_counts)

print("\nBOTTOM 5 — Total Reviews per Listing:")
print(bottom_counts)



TOP 5 — Total Reviews per Listing:
               listing_id  total_review_count
11493            16069685                  57
11591            16842869                  26
14506            38582617                  81
15346            43723586                  62
17982  546505210695128661                  63

BOTTOM 5 — Total Reviews per Listing:
                listing_id  total_review_count
5506   1279389366908702099                   6
6732   1328494094282574396                   5
7551   1354621532086945799                   8
10296  1446867008477628010                   5
10929  1469837763308127204                   5


In [26]:
import pandas as pd

# Load the correct review sample
bottom_reviews_path = r"C:\Users\emman\Documents\Airbnb_Project\results\bottom_5_sampled_reviews.csv"
bottom_reviews = pd.read_csv(bottom_reviews_path)

# Extract unique listing IDs
correct_bottom_ids = bottom_reviews['listing_id'].astype(str).unique().tolist()
correct_bottom_ids


['1279389366908702099',
 '1328494094282574396',
 '1354621532086945799',
 '1446867008477628010',
 '1469837763308127204']

In [29]:
# Load your cleaned listings dataset
clean_listings = pd.read_csv(r"C:\Users\emman\Documents\Airbnb_Project\data\cleaned_listings_df.csv")

# Ensure types match
clean_listings['id'] = clean_listings['id'].astype(str)

# Filter correct rows
correct_bottom_listings = clean_listings[clean_listings['id'].isin(correct_bottom_ids)]

correct_bottom_listings


,id,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,neighborhood_overview,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,bathrooms_text,price,number_of_reviews,review_scores_rating,reviews_per_month
33960,1279389366908702099,The Creative Nest Hub,540076227.0,Four Gems,f,Pomona,Other Cities,NaN,34.072176,-117.743459,Entire home,Entire home/apt,4.0,1.0,2.0,1 bath,$79.00,6.0,3.0,0.65
35601,1328494094282574396,Cozy Studio by Rodeo Drive,660630470.0,Ariella,f,Beverly Hills,Other Cities,"Located in the heart of Beverly Hills, this st...",34.062631,-118.411154,Entire rental unit,Entire home/apt,2.0,1.0,1.0,1 bath,$126.00,5.0,3.4,0.68
36746,1354621532086945799,Cozy 2nd-Floor Studio Sleeps 2,668460439.0,Mohammad,f,Beverly Hills,Other Cities,This studio apartment is perfectly located jus...,34.059543,-118.389206,Entire rental unit,Entire home/apt,2.0,0.0,1.0,1 bath,$108.00,8.0,3.0,1.66
40840,1446867008477628010,Spacious private br and own ba,109414810.0,James,t,Alhambra,Other Cities,NaN,34.089370,-118.125170,Private room in home,Private room,2.0,1.0,1.0,1 private bath,$43.00,5.0,3.2,2.05
42236,1469837763308127204,Tiki Hut Love Shack. 10min walk to everything,366965961.0,Daria,f,Marina del Rey,Unincorporated Areas,NaN,33.976580,-118.452450,Entire rental unit,Entire home/apt,2.0,0.0,1.0,1 bath,$83.00,5.0,3.4,3.85


In [30]:
save_path = r"C:\Users\emman\Documents\Airbnb_Project\results\bottom_5_listings.csv"
correct_bottom_listings.to_csv(save_path, index=False)

print("✔ Corrected bottom_5_listings.csv saved!")


✔ Corrected bottom_5_listings.csv saved!


In [31]:
import os

correct_ids = set(correct_bottom_ids)

existing = []
folder = r"C:\Users\emman\Documents\Airbnb_Project\results\llm_outputs"

for file in os.listdir(folder):
    if file.endswith("_review_analysis.json"):
        listing_id = file.split("_")[1]
        existing.append(listing_id)

missing_ids = correct_ids - set(existing)
missing_ids


{'1328494094282574396', '1446867008477628010', '1469837763308127204'}